In [1]:
import json
import numpy as np
import os
import torch

from human_body_prior.tools.omni_tools import copy2cpu as c2c

In [2]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
DATA_DIR = '../data/h36m'
subject = '1'
action = '2'
subaction = '1'
with open(os.path.join(DATA_DIR, 'annotations', 'Human36M_subject1_data.json'), 'r') as jsonfile:
    data = json.load(jsonfile)
print(data.keys())

dict_keys(['images', 'annotations'])


In [4]:
def beautiful_json(data):
    return json.dumps(data, indent=4, sort_keys=True)

In [5]:
print(beautiful_json(data['images'][0]))

{
    "action_idx": 2,
    "action_name": "Directions",
    "cam_idx": 1,
    "file_name": "s_01_act_02_subact_01_ca_01/s_01_act_02_subact_01_ca_01_000001.jpg",
    "frame_idx": 0,
    "height": 1002,
    "id": 0,
    "subaction_idx": 1,
    "subject": 1,
    "width": 1000
}


In [6]:
# print(len([x for x in data['images'] if x["action_idx"] == 2 ]))

In [7]:
# load SMPL model
with open(os.path.join(DATA_DIR, 'smpl_param', 'Human36M_subject{}_smpl_param.json'.format(subject)), 'r') as jsonfile:
    smpl_data = json.load(jsonfile)

In [8]:
print("# Frames: ", len(smpl_data[action][subaction].keys()))

# Frames:  277


In [9]:
poses = []
betas = []
trans = []
for frame in range(0, len(smpl_data[action][subaction].keys())):
    key = frame * 5
    if str(key) not in smpl_data[action][subaction].keys():
        continue
    pose = smpl_data[action][subaction][str(key)]['pose']
    poses.append(pose)
    shape = smpl_data[action][subaction][str(key)]['shape']
    betas.append(shape)
#     tran = smpl_data[action][subaction][str(key)]['trans']
    trans.append([0, 0, 0]) # for ease of vis
poses = torch.Tensor(poses).to(comp_device)
betas = torch.Tensor(betas).to(comp_device)
trans = torch.Tensor(trans).to(comp_device)

In [10]:
from human_body_prior.body_model.body_model import BodyModel

bm_fname = '../body_models/smplh/neutral/model.npz'
dmpl_fname = '../body_models/dmpls/neutral/model.npz'

num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_fname=bm_fname, num_betas=num_betas, num_dmpls=num_dmpls, dmpl_fname=dmpl_fname).to(comp_device)
faces = c2c(bm.f)

In [11]:
from body_visualizer.tools.vis_tools import render_smpl_params
images = render_smpl_params(bm,
                            {'pose_body': poses[:, 3:66], 
                            'trans': trans,
                            'betas': betas})
images = np.expand_dims(images, axis=(0, 1))

from body_visualizer.tools.vis_tools import imagearray2file
video = imagearray2file(images, "videos/sample-s{}-a{}-sa{}.mp4".format(subject, action, subaction), fps=15)

In [16]:
import trimesh
from body_visualizer.tools.vis_tools import colors
from body_visualizer.mesh.mesh_viewer import MeshViewer
from body_visualizer.mesh.sphere import points_to_spheres
from body_visualizer.tools.vis_tools import show_image

imw, imh = 1000, 1000
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

In [17]:
# body_pose_beta = bm(**{k:v for k,v in body_parms.items() if k in ['pose_body', 'betas']})

body_pose_beta = bm(pose_body=poses[:, 3:66], 
                    betas=betas)
def vis_body_pose_beta(fId = 0):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose_beta.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    show_image(body_image)

# vis_body_pose_beta(fId=100)

In [13]:
# print(smpl_data['2']['1']['0'].keys())

dict_keys(['pose', 'shape', 'trans', 'fitted_3d_pose'])


In [14]:
# for k, v in smpl_data['2']['1']['0'].items():
#     print(k, len(v))

pose 72
shape 10
trans 1
fitted_3d_pose 17
